# Exemplo de extração de dados de páginas web

# https://www.vivareal.com.br/venda/?pagina=5

Este notebook extrai os dados de todos os [imóveis a venda no site VivaReal](https://www.vivareal.com.br/venda/) e escreve os dados extraidos num banco de dados (SQLIte).

![cartao-vivareal.png](cartao-vivareal.png)

Na última verificação, o site tinha 277 páginas de imóveis, 36 imóveis por página. Se deixá-lo rodar integralmente, teremos no final quase 10000 imóveis.

Notebook escrito por Avi Alkalay <<DataScientist@digitalhouse.com>>

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3

In [5]:
# 2 opções de URL
urlBrasil = "https://www.vivareal.com.br/venda/?pagina={npagina}"
urlPinheiros = "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-oeste/pinheiros/apartamento_residencial/?pagina={npagina}"

# Vamos trabalhar com esta:
url=urlBrasil

# Para enganar o site, permutaremos entre 2 opções de assinaturas de browser.
# Peguei de https://developers.whatismybrowser.com/
userAgents=[
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15"
]

dbfile = 'VivaReal.db'
tabela = 'venda'

## Análise inicial de cada elemento da página

Este trecho serve para testarmos a extração de cada elemento desejado, seja nome do imóvel, preço etc. Quando compreendido, movemos o trecho de código finalizado para o bloco mais adiante no notebook, em que o scrapping é feito em lote.

### Passo 1: Traz o HTML do site

Use o `requests` para fazer uma operação HTTP GET e obter o HTML da página.

In [6]:
doc = requests.get(url.format(npagina=1), headers={"User-agent": userAgents[1]})

Vamos verificar o que o webserver retornou olhando só os 700 primeiros bytes (variavel `olhadinha`).

In [4]:
olhadinha=700
print(str(doc.content[:olhadinha]) + '…')

b' <!DOCTYPE html> <html lang="pt-BR"> <head>  <script type="bf948744663da505b3bdeb50-text/javascript"> (function(i,s,o,g,r,a,m){i[\'GoogleAnalyticsObject\']=r;i[r]=i[r]||function(){\n      (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),\n    m=s.getElementsByTagName(o)[0];a.src=g;m.parentNode.insertBefore(a,m)\n  })(window,document,\'script\',\'https://www.google-analytics.com/analytics.js\',\'ga\');\n  ga(\'create\', \'UA-126375-31\', \'auto\'); </script> <script src="https://www.googleoptimize.com/optimize.js?id=GTM-PVK4X33" type="bf948744663da505b3bdeb50-text/javascript"></script>  <script type="bf948744663da505b3bdeb50-text/javascript"> var pageCategory = \'RESULT PAGE\';\n\n  co'…


In [5]:
doc.status_code

200

### Passo 2: Analiza o texto HTML

A biblioteca `BeautifulSoup` tem a capacidade de analizar o HTML entregue pelo site (que não é nada mais que texto corrido, como vimos), e convertê-lo em DOM (document object model). O DOM é o mesmo documento HTML só que todas as tags e sua hierarquia foram identificadas ao ponto de podermos fazer buscas por tags e attributos de seus tags.

In [7]:
analizador = BeautifulSoup(doc.content, 'html.parser')

A variável `analizador` agora contém o DOM do texto retornado pelo web server. Então agora usamos ela para buscar tags específicas.

Usamos o _inspector_ do browser para entender a estrutura do documento que o HTMLeiro da VivaReal concebeu. Descobrimos que os imóveis estão publicados assim:

```html
<div id="12345">
    <div class="js-card-selector">
        ...mais tags sobre o imóvel...
    </div>
</div>

<div id="54321">
    <div class="js-card-selector">
        ...mais tags sobre o imóvel...
    </div>
</div>
```

Cada imóvel está contido dentro de seu respectivo `<div id="....">`, mas não temos nada específico e genérico nesta tag para selecioná-la. Já a tag imediatamente interior, `<div class="js-card-selector">`, é idêntica para cada imóvel e contém a classe `js-card-selector`, o que a torna uma ótima candidata para ser selecionada.

Então abaixo criaremos uma lista chamada `imoveis` que contém todos os trechos DOM que respeitam o nosso filtro: `<div>`s com classe `js-card-selector`.

In [7]:
analizador

 <!DOCTYPE html>
 <html lang="pt-BR"> <head> <script type="bf948744663da505b3bdeb50-text/javascript"> (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
      (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.src=g;m.parentNode.insertBefore(a,m)
  })(window,document,'script','https://www.google-analytics.com/analytics.js','ga');
  ga('create', 'UA-126375-31', 'auto'); </script> <script src="https://www.googleoptimize.com/optimize.js?id=GTM-PVK4X33" type="bf948744663da505b3bdeb50-text/javascript"></script> <script type="bf948744663da505b3bdeb50-text/javascript"> var pageCategory = 'RESULT PAGE';

  const clickstreamTracker = function (rawData) {
    const clickstreamTrackerEndpoint = 'https://tracking.vivareal.com/events/v2';
    const data = JSON.stringify(rawData);

    if (navigator.sendBeacon) {
      navigator.sendBeacon(clickstreamTrackerEndpoint, data);
    } else {
      const ajax = new 

In [8]:
imoveis = analizador.find_all('div', class_="js-card-selector")

Vamos dar uma olhadinha no que tem dentro de cada ítem do array:

In [9]:
for i in imoveis[:5]:
    print(str(i).replace("\n","").replace("   ","")[:200] + "…\n")

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="p…

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="p…

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="p…

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="p…

<div class="js-card-selector"> <article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <

In [10]:
len(imoveis)

36

### Passo 3: Monto receitas para extrair cada dado que desejo

Ótimo. Agora olhando a estrutura de tags e atributos, uso seletores para isolar exatamente o dado que desejo. Após isolá-lo, ainda forço ele a passar por filtros para eliminar inutilidades como espaços (uso `strip()`), prefixos como "R$ " (uso `replace("R$ ","")`) e ajusto o ponto decimal de números (uso `replace(".","")`) para em seguida convertê-los de texto para números (uso `int()`).

É muito importante para mim também obter o ID do imóvel. Como esta informação está num atributo do `<div>` pai, uso o seletor `parent`.

In [11]:
# Título do anúncio
print(imoveis[0].find("span",class_="property-card__title").contents[0].strip())

Apartamento com 3 Quartos à Venda, 101m²


In [12]:
# Endereço
print(imoveis[0].find("span",class_="property-card__address").contents[0].strip())

Rua Francisco Pessoa, 695 - Vila Andrade, São Paulo - SP


In [13]:
imoveis[0].find("div",class_="property-card__price").find("p").contents[0].strip().replace("R$ ","").replace(".","")

'750000'

In [14]:
float(imoveis[0].find("div",class_="property-card__price").find("p").contents[0].strip().replace("R$ ","").replace(".",""))

750000.0

In [15]:
# Preço limpo e transformado em inteiro
print(int(imoveis[0].find("div",class_="property-card__price").find("p").contents[0].strip().replace("R$ ","").replace(".","")))

750000


In [16]:
# Telefone
try:
    print(imoveis[0].find("a",class_="property-card__contact--phone").get('href').replace("tel:","+55"))
except:
    pass

In [41]:
# Amenidades
try:
  print(imoveis[0].find('ul',class_="property-card__amenities").text.strip().replace('...', ''))
except:
  pass

Elevador Piscina Churrasqueira Salão de festas Playground 


In [17]:
# Texto descritivo
try:
    print(imoveis[0].find('ul',class_='property-card__details').text.strip())
except:
    pass

101   m²     3   Quartos     3   Banheiros     2   Vagas


In [18]:
print(int(imoveis[0].find('ul',class_='property-card__details').find('span',class_='property-card__detail-area').text.strip()))

101


In [19]:
# imoveis[0].find('div',class_='property-card__description').contents[0].strip()

# ID do imovel
print(imoveis[0].parent.get('id'))

2476402673


## Extração em massa

Depois da análise acima, podemos criar nosso loop que consome todo o site. Página por página, imóvel por imóvel, dado por dado.

In [8]:
# são 277 páginas de imóveis a venda no total
# paginas=277

# mas a título de exemplo, usaremos somente 20 páginas:
paginas=20

In [9]:
# em rows guardo uma lista temporária de dicts de dados capturados, 1 dict por imóvel
imóveis=[]

for pagina in range(1,paginas+1):
    print("Estou aqui: " + url.format(npagina=pagina))
    
    # pega a página do site pela internet
    doc = requests.get(url.format(npagina=pagina))
    
    # analiza o HTML
    analizador = BeautifulSoup(doc.content, 'html.parser')
    
    # extrai somente a lista de imóveis (em HTML) usando o seletor descoberto no código da página
    imoveis = analizador.find_all('div', class_="js-card-selector")
    
    for unidade in imoveis:
        uni={}
        # extrai dado por dado segundo seus seletores...
        
        # O ID:
        uni['id'] = unidade.parent.get('id')
        
        # O título/nome:
        uni['nome']  = unidade.find("span",class_="property-card__title").contents[0].strip()
        
        # O endereço:
        uni['ende']  = unidade.find("span",class_="property-card__address").contents[0].strip()
        
        # A metragem
        uni['metragem']=int(unidade.find('ul',class_='property-card__details').find('span',class_='property-card__detail-area').text.strip())
        
        # O preço tem uma pegadinha
        try:
            # Na maioria dos anuncios o preço está na posição 0.
            # Mas em alguns, o preço aparece como "a partir de" e causa um erro aqui.
            uni['preco'] = int(unidade.find("div",class_="property-card__price").find("p").contents[0].strip().replace("R$ ","").replace(".",""))
        except ValueError:
            # Então o meu tratamento de erro para preços "a partir de" é tentar capturá-lo na posição 2:
            try:
                uni['preco'] = int(unidade.find("div",class_="property-card__price").contents[2].strip().replace("R$ ","").replace(".",""))
            except IndexError:
                # Mas há um outro tipo de problema, onde o preço aparece "sob consulta", e causa um "IndexError". Trato assim:
                uni['preco'] = -1

        # O telefone capturo manipulando a URL e dando uma internacionalizada básica com "+55":
        try:
            uni['tel'] = unidade.find("a",class_="property-card__contact--phone").get('href').replace("tel:","+55")
        except:
            pass
        
        # A descrição:
        try:
            uni['desc'] = unidade.find('ul',class_='property-card__details').text.strip()
        except:
            pass
        
        # Amenidades
        try:
          uni['amenidades'] = unidade.find('ul',class_="property-card__amenities").text.strip().replace('...', '')
        except:
          pass

        # No final deste loop, o dict uni contém os dados de 1 imóvel, aí adiciono-o a uma lista de imóveis
        imóveis.append(uni)

Estou aqui: https://www.vivareal.com.br/venda/?pagina=1
Estou aqui: https://www.vivareal.com.br/venda/?pagina=2
Estou aqui: https://www.vivareal.com.br/venda/?pagina=3
Estou aqui: https://www.vivareal.com.br/venda/?pagina=4
Estou aqui: https://www.vivareal.com.br/venda/?pagina=5
Estou aqui: https://www.vivareal.com.br/venda/?pagina=6
Estou aqui: https://www.vivareal.com.br/venda/?pagina=7
Estou aqui: https://www.vivareal.com.br/venda/?pagina=8
Estou aqui: https://www.vivareal.com.br/venda/?pagina=9
Estou aqui: https://www.vivareal.com.br/venda/?pagina=10
Estou aqui: https://www.vivareal.com.br/venda/?pagina=11
Estou aqui: https://www.vivareal.com.br/venda/?pagina=12
Estou aqui: https://www.vivareal.com.br/venda/?pagina=13
Estou aqui: https://www.vivareal.com.br/venda/?pagina=14
Estou aqui: https://www.vivareal.com.br/venda/?pagina=15
Estou aqui: https://www.vivareal.com.br/venda/?pagina=16
Estou aqui: https://www.vivareal.com.br/venda/?pagina=17
Estou aqui: https://www.vivareal.com.br/

Processei todos os imóveis de todas as páginas. Vamos ver o resultado...

In [10]:
imóveis

[{'amenidades': 'Elevador Piscina Churrasqueira Salão de festas Playground ',
  'desc': '101   m²     3   Quartos     3   Banheiros     2   Vagas',
  'ende': 'Rua Francisco Pessoa, 695 - Vila Andrade, São Paulo - SP',
  'id': '2476402673',
  'metragem': 101,
  'nome': 'Apartamento com 3 Quartos à Venda, 101m²',
  'preco': 750000},
 {'amenidades': 'Elevador Piscina Churrasqueira Salão de festas Academia ',
  'desc': '107   m²     3   Quartos     2   Banheiros     2   Vagas',
  'ende': 'Rua Paris, 241 - Sumaré, São Paulo - SP',
  'id': '2473758281',
  'metragem': 107,
  'nome': 'Apartamento com 3 Quartos à Venda, 107m²',
  'preco': 850000},
 {'amenidades': 'Mobiliado Elevador Piscina Área de serviço Churrasqueira ',
  'desc': '70   m²     2   Quartos     2   Banheiros     1   Vaga',
  'ende': 'Rua Cipriano Barata, 790 - Ipiranga, São Paulo - SP',
  'id': '2475837894',
  'metragem': 70,
  'nome': 'Apartamento com 2 Quartos à Venda, 70m²',
  'preco': 565000},
 {'amenidades': 'Mobiliado Ele

Agora converterei minha lista de dicts para um DataFrame chamado `todosOsImoveis`. Para tal, crio um DataFrame vazio nomeando as colunas com as chaves de um dict `uni`.

In [11]:
# Crio um DataFrame vazio cujas colunas são as chaves de 1 unidade:
todosOsImoveis=pd.DataFrame(columns=uni.keys())
todosOsImoveis

,id,nome,ende,metragem,preco,desc,amenidades


Com o esqueleto do DataFrame pronto, adiciono nele todos os meus dicts (imóveis).

In [12]:
todosOsImoveis=todosOsImoveis.append(imóveis)

todosOsImoveis.head()

,id,nome,ende,metragem,preco,desc,amenidades
0,2476402673,"Apartamento com 3 Quartos à Venda, 101m²","Rua Francisco Pessoa, 695 - Vila Andrade, São ...",101,750000,101 m² 3 Quartos 3 Banheiros ...,Elevador Piscina Churrasqueira Salão de festas...
1,2473758281,"Apartamento com 3 Quartos à Venda, 107m²","Rua Paris, 241 - Sumaré, São Paulo - SP",107,850000,107 m² 3 Quartos 2 Banheiros ...,Elevador Piscina Churrasqueira Salão de festas...
2,2475837894,"Apartamento com 2 Quartos à Venda, 70m²","Rua Cipriano Barata, 790 - Ipiranga, São Paulo...",70,565000,70 m² 2 Quartos 2 Banheiros ...,Mobiliado Elevador Piscina Área de serviço Chu...
3,2476743581,"Apartamento com 2 Quartos à Venda, 57m²","Rua Cônego Vicente Miguel Marino, 183 - Barra ...",57,490000,57 m² 2 Quartos 2 Banheiros ...,Mobiliado Elevador Piscina Churrasqueira Salão...
4,2477112488,"Apartamento com 6 Quartos à Venda, 290m²","Alameda Tietê, 325 - Jardim Paulista, São Paul...",290,3000000,290 m² 6 Quartos 5 Banheiros ...,Elevador Academia Condomínio fechado


Agora em Pandas, vamos dar uma otimizadinha básica convertendo algumas colunas textuais para tipos numéricos.

In [51]:
todosOsImoveis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 719
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          720 non-null    object
 1   nome        720 non-null    object
 2   ende        720 non-null    object
 3   metragem    720 non-null    object
 4   preco       720 non-null    object
 5   desc        720 non-null    object
 6   amenidades  660 non-null    object
dtypes: object(7)
memory usage: 45.0+ KB


Como ficaram os tipos de nossas colunas...

In [13]:
todosOsImoveis=todosOsImoveis.convert_dtypes()

In [14]:
todosOsImoveis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 719
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          720 non-null    string
 1   nome        720 non-null    string
 2   ende        720 non-null    string
 3   metragem    720 non-null    Int64 
 4   preco       720 non-null    Int64 
 5   desc        720 non-null    string
 6   amenidades  660 non-null    string
dtypes: Int64(2), string(5)
memory usage: 46.4 KB


Por que a coluna `id` não foi convertida para `Int64` como as outras? Vamos forçar:

In [15]:
todosOsImoveis['id']=pd.to_numeric(todosOsImoveis['id'])

In [16]:
todosOsImoveis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 719
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          720 non-null    int64 
 1   nome        720 non-null    string
 2   ende        720 non-null    string
 3   metragem    720 non-null    Int64 
 4   preco       720 non-null    Int64 
 5   desc        720 non-null    string
 6   amenidades  660 non-null    string
dtypes: Int64(2), int64(1), string(4)
memory usage: 46.4 KB


Seta o índice para o ID do imóvel

In [17]:
todosOsImoveis.set_index(keys='id', inplace=True)

Eis o nosso DataFrame pronto:

In [18]:
todosOsImoveis.head()

,nome,ende,metragem,preco,desc,amenidades
id,,,,,,
2476402673,"Apartamento com 3 Quartos à Venda, 101m²","Rua Francisco Pessoa, 695 - Vila Andrade, São ...",101,750000,101 m² 3 Quartos 3 Banheiros ...,Elevador Piscina Churrasqueira Salão de festas...
2473758281,"Apartamento com 3 Quartos à Venda, 107m²","Rua Paris, 241 - Sumaré, São Paulo - SP",107,850000,107 m² 3 Quartos 2 Banheiros ...,Elevador Piscina Churrasqueira Salão de festas...
2475837894,"Apartamento com 2 Quartos à Venda, 70m²","Rua Cipriano Barata, 790 - Ipiranga, São Paulo...",70,565000,70 m² 2 Quartos 2 Banheiros ...,Mobiliado Elevador Piscina Área de serviço Chu...
2476743581,"Apartamento com 2 Quartos à Venda, 57m²","Rua Cônego Vicente Miguel Marino, 183 - Barra ...",57,490000,57 m² 2 Quartos 2 Banheiros ...,Mobiliado Elevador Piscina Churrasqueira Salão...
2477112488,"Apartamento com 6 Quartos à Venda, 290m²","Alameda Tietê, 325 - Jardim Paulista, São Paul...",290,3000000,290 m² 6 Quartos 5 Banheiros ...,Elevador Academia Condomínio fechado


##Removendo valores duplicados 

No momento em que estou rodando esse código temos 683 valores duplicados, isto é, imóveis repetidos.

In [19]:
todosOsImoveis.duplicated().sum()

683

Vamos cortar esses valores do dataframe antes de inseri-los em um banco de dados sqlite

In [20]:
todosOsImoveis.drop_duplicates(inplace= True)

Conferindo se foram retirados

In [21]:
todosOsImoveis.duplicated().sum()

0

## Grava o DataFrame inteiro num Banco de Dados

Este é o código para abrir ou criar um aquivo SQLite chamado `VivaReal.db`, na tabela `venda`. Para usar outro banco, como MariaDB, Oracle, DB2, altere esta célula e use a biblioteca SQL Alchemy com os drivers corretos. Todo o resto do código funcionará igual.

In [22]:
dbfile = 'VivaReal.db'
tabela = 'venda'

db = sqlite3.connect(dbfile)

Determina os tipos de cada coluna SQL baseado nos tipos das colunas Pandas:

In [23]:
sqlDataTypes={}

for c in todosOsImoveis.columns:
    if todosOsImoveis[c].dtype.kind == 'i':
        sqlDataTypes[c]='INTEGER'
    elif todosOsImoveis[c].dtype.kind == 'f':
        sqlDataTypes[c]='REAL'
    else:
        sqlDataTypes[c]='TEXT'

sqlDataTypes

{'amenidades': 'TEXT',
 'desc': 'TEXT',
 'ende': 'TEXT',
 'metragem': 'INTEGER',
 'nome': 'TEXT',
 'preco': 'INTEGER'}

Escreve na tabela

In [24]:
todosOsImoveis.to_sql(tabela, index=True, if_exists='replace', dtype=sqlDataTypes, con=db)

Efetiva as operações no DB e fecha o arquivo

In [25]:
db.commit()
db.close()

## Checa os dados no DB

In [80]:
dbfile = 'VivaReal.db'
tabela = 'venda'

db = sqlite3.connect(dbfile)

imoveisCaros = pd.read_sql_query(f'select * from "{tabela}" where preco>600000', db)

imoveisCaros

,id,nome,ende,metragem,preco,desc
0,2476402673,"Apartamento com 3 Quartos à Venda, 101m²","Rua Francisco Pessoa, 695 - Vila Andrade, São ...",101,750000,101 m² 3 Quartos 3 Banheiros ...
1,2473758281,"Apartamento com 3 Quartos à Venda, 107m²","Rua Paris, 241 - Sumaré, São Paulo - SP",107,850000,107 m² 3 Quartos 2 Banheiros ...
2,2477112488,"Apartamento com 6 Quartos à Venda, 290m²","Alameda Tietê, 325 - Jardim Paulista, São Paul...",290,3000000,290 m² 6 Quartos 5 Banheiros ...
3,2473759712,"Apartamento com 3 Quartos à Venda, 160m²","Rua André Dreyfus, 109 - Sumaré, São Paulo - SP",160,1300000,160 m² 3 Quartos 3 Banheiros ...
4,2476277377,"Apartamento com Quarto à Venda, 84m²","Rua Turiassu, 507 - Perdizes, São Paulo - SP",84,1060000,84 m² 1 Quarto 2 Banheiros 1...
...,...,...,...,...,...,...
275,2494214252,"Apartamento com 3 Quartos à Venda, 77m²","Rua Artur Thiré - Vila da Saúde, São Paulo - SP",77,680000,77 m² 3 Quartos 3 Banheiros ...
276,2494336886,"Casa de condomínio com 4 Quartos à Venda, 341m²","Rua Capitão Alberto Mendes Júnior, 31 - Loteam...",341,1680000,341 m² 4 Quartos 6 Banheiros ...
277,2499833175,"Apartamento com 3 Quartos à Venda, 162m²","Avenida José Medeiros Vieira, 1876 - Praia Bra...",162,1740000,162 m² 3 Quartos 4 Banheiros ...
278,2498679480,"Apartamento com 2 Quartos à Venda, 83m²","Rua Raposo Tavares, 102 - Jardim das Acácias, ...",83,680000,83 m² 2 Quartos 3 Banheiros ...


## Sincronização Diária

Para manter os dados em dia, este procedimento deve ser executado diariamente.

No final da extração comparo todos os imóveis da minha base histórica com todos os imóveis que acabei de extrair novamente. ID por ID. É por isso que é tão importante se prender aos IDs. É neste momento que detecto qual imóvel sumiu da base, qual apareceu, qual mudou de preço ou descrição.

* Imóveis que estão na minha base histórica mas não estão na última extração serão marcados como deletados ou “unlisted”.
* Imóveis que não estavam na minha base histórica e que acabaram de aparecer na última extração serão marcados como novos.
* Imóveis que aprentam preços diferentes entre a base histórica e a última extração devem ter essa alteração registrada em outra tabela. É interessante manter esse histórico de alteração de preços de imóveis.

Certamente nossa modelagem simples de tabela não suporta registrar a mudança de preços. **Fica de lição** de casa criar uma tabela separada de preços de imóveis para que se possa manter um histórico.

## Exercício

Melhorar o código e extrair também a lista de amenidades (elevador, piscina, churrasqueira etc)